### Importations et configuration du chemin racine du projet

In [32]:
import pandas as pd  # pour la manipulation de DataFrames
import json           # pour lire et écrire des fichiers JSON
import os             # pour gérer les chemins et interactions système
import yaml           # pour lire et écrire des fichiers YAML
from pathlib import Path  # pour manipuler les chemins de fichiers de manière portable
from typing import Dict   # pour typer les dictionnaires dans les fonctions

# Définir le chemin racine du projet
# Ici on prend le dossier parent du répertoire courant
PROJECT_ROOT = Path.cwd().parent

# Afficher le chemin racine pour vérification
PROJECT_ROOT


PosixPath('/home/kaouter/Bureau/projects/briefs_simplon/brief2/Brief-2-ETL-de-donn-es-footballistiques-Wickets-Sprinters')

### Fonctions utilisées pour l’extraction de données

In [33]:


def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Objectif :
        Charger les paramètres de configuration depuis un fichier YAML.
    Paramètres :
        config_filename (str) : Chemin relatif ou absolu du fichier YAML.
    Retour :
        dict : Paramètres de configuration.
    """
    config_path = Path(config_filename)

    # Résoudre le chemin relatif depuis la racine du projet
    if not config_path.is_absolute():
        project_root = Path(__file__).resolve().parents[1]
        config_path = project_root / config_path

    if not config_path.exists():
        raise FileNotFoundError(f"Fichier de configuration introuvable : {config_path}")

    # Charger le fichier YAML
    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config


def fct_read_csv(root_file: str) -> pd.DataFrame:
    """
    Objectif :
        Lire un fichier CSV et retourner un DataFrame pandas.
        Détecte automatiquement le séparateur.
    Paramètres :
        root_file (str) : Chemin relatif ou absolu du fichier CSV.
    Retour :
        pd.DataFrame : Les données du CSV, ou DataFrame vide si fichier introuvable.
    """
    seps = [',', ';', '|', '\t']
    file_path = Path(root_file)

    # Résoudre le chemin relatif
    if not file_path.is_absolute():
        try:
            project_root = Path(__file__).resolve().parents[1]
        except NameError:  # Cas notebook Jupyter
            project_root = Path.cwd().parents[1]
        file_path = project_root / file_path

    if not file_path.exists():
        print(f"Erreur : fichier {file_path} introuvable")
        return pd.DataFrame()

    # Essayer chaque séparateur
    for sep in seps:
        try:
            df = pd.read_csv(file_path, sep=sep, encoding="utf-8", skipinitialspace=True)
            if df.shape[1] > 1:  # Séparateur correct
                return df
        except Exception:
            continue

    print(f"Aucun séparateur valide trouvé pour {file_path}")
    return pd.DataFrame()


def fct_read_json_nested(root_file: str) -> Dict[str, pd.DataFrame]:
    """
    Objectif :
        Charger un JSON imbriqué (teams, stadiums, tvchannels, groups, rounds, matches)
        dans des DataFrames séparés.
    Paramètres :
        root_file (str) : Chemin du fichier JSON.
    Retour :
        dict : Dictionnaire de DataFrames pandas :
            - 'teams', 'stadiums', 'tvchannels', 'groups', 'rounds', 'matches', 'bridge_match_channels'
    """
    with open(root_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    dfs = {}
    # Entités simples
    dfs['teams'] = pd.DataFrame(data.get('teams', []))
    dfs['stadiums'] = pd.DataFrame(data.get('stadiums', []))
    dfs['tvchannels'] = pd.DataFrame(data.get('tvchannels', []))

    # Groupes
    groups_rows = []
    for group_id, group_data in data.get('groups', {}).items():
        groups_rows.append({
            'group_id': group_id,
            'group_name': group_data.get('name'),
            'winner_team_id': group_data.get('winner'),
            'runnerup_team_id': group_data.get('runnerup')
        })
    dfs['groups'] = pd.DataFrame(groups_rows)

    # Matchs et liens match-channel
    matches_rows = []
    match_channels_rows = []

    # Phase de groupes
    for group_id, group_data in data.get('groups', {}).items():
        for match in group_data.get('matches', []):
            match_id = match.get('name')
            matches_rows.append({
                'match_id': match_id,
                'type': match.get('type'),
                'stage': 'group',
                'group_id': group_id,
                'round_id': None,
                'date': match.get('date'),
                'stadium_id': match.get('stadium'),
                'home_team_id': match.get('home_team'),
                'away_team_id': match.get('away_team'),
                'home_result': match.get('home_result'),
                'away_result': match.get('away_result'),
                'home_penalty': None,
                'away_penalty': None,
                'winner': match.get('winner'),
                'finished': match.get('finished'),
                'matchday': match.get('matchday'),
                'channels': match.get('channels', [])
            })
            for channel_id in match.get('channels', []):
                match_channels_rows.append({'match_id': match_id, 'channel_id': channel_id})

    # Phase à élimination directe
    rounds_rows = []
    for round_key, round_data in data.get('knockout', {}).items():
        rounds_rows.append({'round_id': round_key, 'round_name': round_data.get('name')})
        for match in round_data.get('matches', []):
            match_id = match.get('name')
            matches_rows.append({
                'match_id': match_id,
                'type': match.get('type'),
                'stage': 'knockout',
                'group_id': None,
                'round_id': round_key,
                'date': match.get('date'),
                'stadium_id': match.get('stadium'),
                'home_team_id': match.get('home_team'),
                'away_team_id': match.get('away_team'),
                'home_result': match.get('home_result'),
                'away_result': match.get('away_result'),
                'home_penalty': match.get('home_penalty'),
                'away_penalty': match.get('away_penalty'),
                'winner': match.get('winner'),
                'finished': match.get('finished'),
                'matchday': match.get('matchday'),
                'channels': match.get('channels', [])
            })
            for channel_id in match.get('channels', []):
                match_channels_rows.append({'match_id': match_id, 'channel_id': channel_id})

    # Convertir en DataFrames
    dfs['matches'] = pd.DataFrame(matches_rows)
    dfs['bridge_match_channels'] = pd.DataFrame(match_channels_rows)
    dfs['rounds'] = pd.DataFrame(rounds_rows)

    return dfs


def fct_add_prefix_to_df(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """
    Objectif :
        Ajouter un préfixe à toutes les colonnes d'un DataFrame.
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        prefix (str) : Préfixe à ajouter.
    Retour :
        pd.DataFrame : DataFrame avec les noms de colonnes modifiés.
    """
    for col in df.columns:
        df.rename(columns={col: f"{prefix}_{col}"}, inplace=True)
    return df


def fct_extract_data(
    root_csv_2010: str,
    root_csv_2014: str,
    root_csv_2022: str,
    root_json_2018: str
) -> dict:
    """
    Objectif :
        Extraire les données de plusieurs fichiers CSV et JSON et retourner 
        un dictionnaire de DataFrames.
    
    Paramètres :
        root_csv_2010 (str) : Chemin vers le fichier CSV 2010.
        root_csv_2014 (str) : Chemin vers le fichier CSV 2014.
        root_csv_2022 (str) : Chemin vers le fichier CSV 2022.
        root_json_2018 (str) : Chemin vers le fichier JSON 2018.
    
    Retour :
        dict : Dictionnaire contenant les DataFrames extraites.
               Pour le JSON, plusieurs DataFrames (matches, teams, groups, etc.).
    """
    # Lire les fichiers CSV
    df_2010 = fct_read_csv(root_csv_2010)
    df_2014 = fct_read_csv(root_csv_2014)
    df_2022 = fct_read_csv(root_csv_2022)

    # Lire le fichier JSON 2018
    dfs_2018 = fct_read_json_nested(root_json_2018)

    # Afficher un aperçu des données pour vérification
    print("CSV 2010 :")
    print(df_2010.head())
    print("CSV 2014 :")
    print(df_2014.head())
    print("CSV 2022 :")
    print(df_2022.head())
    print("JSON 2018 - Matches :")
    print(dfs_2018['matches'].head())

    # Retourner les données dans un dictionnaire
    return {
        'csv_2010': df_2010,
        'csv_2014': df_2014,
        'csv_2022': df_2022,
        'json_2018': dfs_2018
    }


In [34]:
#Récupération de la configuration
config_path = os.path.join(Path.cwd().parent, 'config.yaml')
config = fct_load_config(config_path)

# Extract

### Extraction à partir du fichier : "data_2018.json"

In [35]:
# Construire le chemin absolue vers le fichier data_2018.json
root_json_2018 = config['root_json_2018']
json_path = PROJECT_ROOT / root_json_2018
dfs_2018 = fct_read_json_nested(json_path)

In [36]:
# Extraction des DataFrames individuels

# Extraction des données staduims
df_stadiums = dfs_2018['stadiums']

# Extraction des données tvchannels
df_tvchannels = dfs_2018['tvchannels']

# Extraction des données teams
df_teams = dfs_2018['teams']

# Extraction des données groupes
df_groups= dfs_2018['groups']

# Extraction des données rounds
df_rounds = dfs_2018['rounds']

# Extraction des données matches
df_matches = dfs_2018['matches']


### Visualisation des dataframes

In [37]:

# # Dataframe staduims
# df_stadiums.head()

# # Dataframe tvchannels
# df_tvchannels.head() 
   
# # Dataframe teams
# df_teams.head()

# # Dataframe groupes
# df_groups.head()

# # Dataframe rounds
# df_rounds.head()

# Dataframe matches
df_matches.head()

,match_id,type,stage,group_id,round_id,date,stadium_id,home_team_id,away_team_id,home_result,away_result,home_penalty,away_penalty,winner,finished,matchday,channels
0,1,group,group,a,None,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,True,1,"[4, 6, 13, 17, 20, 22]"
1,2,group,group,a,None,2018-06-15T17:00:00+05:00,12,3,4,0,1,NaN,NaN,NaN,True,1,"[3, 6, 14, 17, 20, 22]"
2,17,group,group,a,None,2018-06-19T21:00:00+03:00,3,1,3,3,1,NaN,NaN,NaN,True,2,"[3, 6, 13, 17, 15, 21, 22]"
3,18,group,group,a,None,2018-06-20T18:00:00+03:00,10,4,2,1,0,NaN,NaN,NaN,True,2,"[3, 6, 13, 17, 21, 22]"
4,33,group,group,a,None,2018-06-25T18:00:00+04:00,7,4,1,3,0,NaN,NaN,NaN,True,3,"[4, 6, 13, 18, 15, 20, 22]"
